**Project 2**  
*AEEM 2058*  
*Date*:11/30/2025  
Hayden Mays, Logan Mueller, Evan Wills  
___

In [54]:
# Haydens Version
# Imports
import numpy as np
import time

---

In [156]:
# A)
t_initial=0
t_finial=10
t_step=0.01
N=int((t_finial-t_initial)/t_step)

def Euler(t_0:float,t_f:float,h:float,S_initial:float,dSdt:callable)-> tuple [np.ndarray:'S values',np.ndarray]:
    ''' Runs Euler method
    
    Args:
        t_0: the initial time
        t_f: the finial time
        h: the step length between the initial and finial time
        S_initial: the initial condition
        dSdt: the ODE for S
        
    Returns:
        tuple of 2 arrays
        (t_span,S)
        
    '''
    t_span=np.arange(t_0,t_f+h,h) # creats span
    
    S=np.zeros((len(t_span),2)) # Creates array for outputs
    S[0]=S_initial #Sets intial value
    
    # Runs euler aproximation
    for i in range(len(t_span)-1):
        S[i+1]=S[i]+h*dSdt(t_span[i],S[i])
    
    # Returns t_span and the outputs
    return t_span,S

def system(t,state):
    v_x,v_z=state
    g=32.2 #ft/s^2
    c=10e-3
    r=np.sqrt(v_x**2+v_z**2)
    dv_x=-c*v_x*r
    dv_z=-g-c*v_z*r
    return np.array([dv_x,dv_z])

v0=8000 #ft/s
theta0=80

y0=np.array([v0*np.cos(np.deg2rad(theta0)),v0*np.sin(np.deg2rad(theta0))])

t_c,v=Euler(t_initial,t_finial,t_step,y0,system)
v_x=v.T[0]
trapx=(t_step/2)*(v_x[0]+ 2*sum(v_x[1:N-1])+v_x[N-1])

v_z=v.T[1]
trapz=(t_step/2)*(v_z[0]+ 2*sum(v_z[1:N-1])+v_z[N-1])

print(f'Initial Velocity: {v0:0.03f} ft/s')
print(f'Initial Angle: {theta0:0.03f}°\n')

print('Time'.center(10)+'|'+'X Velocity'.center(15)+'|'+'Y Velocity'.center(14))
print(''.center(42,'-'))
for t,vx,vz in zip(t_c,v_x,v_z):
    print(f'{t:7.03f} s | {vx:8.03f} ft/s | {vz:8.03f} ft/s'.center(42),end='\r',flush=True)
    time.sleep(t_step)

print(f'\n\nRange: {trapx:0.03f} ft | {trapx/5280:0.03f} Mile(s)')

print(f'Finial Z: {trapz:0.03f} ft')

Initial Velocity: 8000.000 ft/s
Initial Angle: 80.000°

   Time   |   X Velocity  |  Y Velocity  
------------------------------------------
 10.000 s |    0.281 ft/s |  -56.716 ft/s 

Range: 100.097 ft | 0.019 Mile(s)
Finial Z: 12.352 ft


This is what I got so far, this is forward solving it, while the question needs us to back solve for velocity and angle

In [56]:
# Validation

---

In [57]:
# B)

Discusion

---

In [58]:
# C) 😦